<a href="https://colab.research.google.com/github/wordsbyanmol/Machine-Learning-Course/blob/main/Anmol_mlpp24_assignment_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MLPP24 // Assignment #4**

assigned : **Mar 7, 2024**

DUE : **Mar 22, 2024 11:59pm**

## Outcomes of gentrification: local changes to median income

<img src="https://cdn.theatlantic.com/assets/media/img/mt/2015/08/20720085886_db5d1f89e8_o/lead_720_405.jpg?mod=1533691764" width=500>

---

<b>

How to complete and submit assignments:

1. Please make a copy of this notebook in your Google Drive and add your name to the filename.

2. Once you have completed the notebook, please share it with me before the due date and time by clicking the "Share" button in the upper right corner of the notebook.


Rules for assignments:

1. You may work with other students in the class, but if you do, each student with whom you worked <u>must</u> be listed below.  Direct copying from someone else's notebook is not permitted.

2. You may use generative AI models (e.g., ChatGPT) to help complete the assignment but if you do you must answer YES to the question below and bear in mind that such models <u>often</u> yield incorrect and biased solutions.

3. All solutions and outputs must be derived with python and the notebook should be "runable" by me (top to bottom) without errors.

4. Late assignments will assess a 15% late penalty up to 3 days after the due date and a 50% late penalty until the end of the term.

</b>

<u>**Instructions for tasks that will be graded are in bold below.**</u>

---

**Please list the names of the other students with whom you worked for this assignment (if none, put "None").**

TEXT FOR YOUR ANSWER HERE

**Did you use a generative AI model (e.g., ChatGPT) to create text or code for this assignment?**

Yes

---

In class we discussed the definition of gentrification and the outcomes associated with that process.  Our specific example was the changing demographic makeup of Philadelphia over the period of 2000 to 2016.  In this assignment, we'll be doing an identical comparison, but considering local changes to median income.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **PART 1 - Loading and Working With the Census Data**


<b> Link your google drive and install geopandas. </b>

In [ ]:
#import libraries for analysis
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import geopandas as gp

<b> Load in the pre-cleaned 2000 median income data from the `data/census/income/` directory <u>as well as</u> the 2000 and 2016 shapefiles (in the `data/census/ctshapes` directory).  Make sure that you call the census tract GeoDataFrames `ct00` and `ct16`.</b>

In [ ]:
#load data from Drive
fname= "/content/drive/Shareddrives/mlpp24/data/census/income/census_income_2000.csv"

median_income= pd.read_csv(fname)
median_income_copy= median_income
median_income

<b>

Load the 2016 ACS 5-year estimates for the median income for census tracts from `data/census/income/` and drop all rows for which the median income (column `"B19013_001E"`) is equal to `-666666666`.

</b>

In [ ]:
#load data from Drive
fname2016= "/content/drive/Shareddrives/mlpp24/data/census/income/census_income_2016.csv"

acs16= pd.read_csv (fname2016)
acs16

In [ ]:
# Count the occurrences of each value in the column
value_counts = acs16['B19013_001E'].value_counts()
value_counts

if -666666666 in value_counts:
    print(f"Number of rows with value -666666666: {value_counts[-666666666]}")
else:
    print("There are no rows with value -666666666 in the specified column.")

In [ ]:
# Drop rows with median income equal to -666666666
acs16= acs16[acs16['B19013_001E'] != -666666666]

# Display the dataframe
print(acs16)

<b> In the income DataFrames, rename the column `"Median household income in 1999"` to `"income_2000"` and the column `"B19013_001E"` to `"income_2016"` (print the heads of the DataFrames to see if it worked).</b>

In [ ]:
# Rename columns
median_income.columns = ['Id2_2000', 'income_2000']

# Display the DataFrame with renamed columns
median_income. head ()

In [ ]:
# Rename columns
acs16.columns =['income_2016', 'GEOID']

# Display the DataFrame with renamed columns
acs16

In order to compare 2000 dollars to 2016 dollars we need to adjust for inflation.  We could assume a model for inflation, but instead, let's just adjust the 2000 data so that it has the same mean as the 2016 data.

<b> First, plot a histogram of the 2000 incomes and a histogram of the 2016 incomes (on the same plot) each with 50 bins and range 0 to 140000 and `alpha=0.5`. </b>

In [ ]:
# Plot histograms for incomes in 2000 and 2016 on the same plot
plt.hist(median_income["income_2000"], bins=50, range=(0, 140000), alpha=0.5, label='2000')
plt.hist(acs16["income_2016"], bins=50, range=(0, 140000), alpha=0.5, label='2016')

# Customize the plot
plt.xlabel('Income')
plt.ylabel('Frequency')
plt.title('Median Income for 2000 and 2016')
plt.legend()

# Show the plot
plt.show()

You can see that the 2016 incomes are a bit higher on average.

<b> Now calculate the ratio between the mean 2016 income and mean 2000 income.  Create a column in the 2000 income DataFrame called `"income_2000_adj"` that is `"income_2000"` times that ratio.  Print the mean of `"income_2016"` and `"income_2000_adj"` to make sure that they are now the same and replot the histograms above with the 2016 and 2000 adjusted incomes. </b>

In [ ]:
# Calculate the mean income for 2016
mean_income_2016 = acs16['income_2016'].mean()

# Calculate the mean income for 2000
mean_income_2000 = median_income['income_2000'].mean()

# Calculate the ratio
income_ratio = mean_income_2016 / mean_income_2000

print("Mean income in 2016:", mean_income_2016)
print("Mean income in 2000:", mean_income_2000)
print("Income ratio (2016/2000):", income_ratio)

In [ ]:
#Create a column in the 2000 income DataFrame called "income_2000_adj"
median_income["income_2000_adj"] = median_income["income_2000"]* income_ratio
median_income. head ()

In [ ]:
# Calculate the mean of income_2016 column
mean_income_2016 = acs16["income_2016"].mean()

# Calculate the mean of income_2000_adj column
mean_income_2000_adj = median_income["income_2000_adj"].mean()

# Print the means
print(mean_income_2016)
print(mean_income_2000_adj)

In [ ]:
median_income. head ()

In [ ]:
# Plot histograms for incomes in 2000 and 2016 on the same plot
plt.hist(median_income["income_2000_adj"], bins=50, range=(0, 140000), alpha=0.5, label='2000')
plt.hist(acs16["income_2016"], bins=50, range=(0, 140000), alpha=0.5, label='2016')

# Customize the plot
plt.xlabel('Income')
plt.ylabel('Frequency')
plt.title('Median Incomes for 2000 and 2016')
plt.legend()

# Show the plot
plt.show()

The 2000 incomes have now been adjusted to 2016 levels$^{\dagger}$ so that we can now take meaningful differences.

<small>$^{\dagger}$ under several assumptions! </small>

### **PART 2 - Merging and Visualizing the Data**

Now we'd like to make some maps of the income data.  Before we do that, we need to clean up the census tract GeoDataFrames a bit.

<b>As we did in class, create a proper `"GEOID"` for year 2000 and convert both year 2000 and year 2016 `"GEOID"`s into integers.</b>




In [ ]:
#load files
fname00= "/content/drive/Shareddrives/mlpp24/data/census/ctshapes/2000/tr42_d00.shp"
fname16= "/content/drive/Shareddrives/mlpp24/data/census/ctshapes/2016/cb_2016_42_tract_500k.shp"

In [ ]:
#name files
ct00 = gp.read_file(fname00)
ct16 = gp.read_file(fname16)

In [ ]:
median_income. head ()

In [ ]:
ct00.plot ()

In [ ]:
ct00

In [ ]:
unique_states = ct00["STATE"].unique()
print(unique_states)

In [ ]:
#string manipulation
# Concatenate the values of the three columns into a new column

state = ct00['STATE']
county= ct00['COUNTY']
tract = ct00["TRACT"]
tract= tract + "00"
tract= tract.str[:6]
geoid= state+county+tract
ct00["GEOID"]= geoid

# Display the DataFrame with the new column
ct00

In [ ]:
ct00["GEOID"].info ()

In [ ]:
# Convert the new column to integers
ct00['GEOID'] = ct00['GEOID'].astype(int)
ct00. info ()

In [ ]:
ct16.head ()

In [ ]:
# Convert column to integers
ct16['GEOID'] = ct16['GEOID'].astype(int)
ct16. info ()

<b> Now, merge the 2000 income data into the 2000 census tract GeoDataFrame, creating a new GeoDataFrame called `inc00sh`, and merge the 2016 income data into the 2016 census tract GeoDataFrame creating a new GeoDataFrame called `inc16sh`.</b>

In [ ]:
median_income. head(2)

In [ ]:
ct00. head (2)

In [ ]:
#merge dataframe
inc00sh= ct00.merge(median_income, left_on= "GEOID", right_on= "Id2_2000")
inc00sh. head (3)

In [ ]:
inc00sh["COUNTY"].value_counts ()

In [ ]:
na_count = inc00sh['income_2000_adj'].isna().sum()
print("Number of NaN values in 'income_2000_adj' column:", na_count)

# Remove NaN values from a specific column
inc00sh = inc00sh.dropna(subset=['income_2000_adj'])
inc00sh. head (3)


In [ ]:
# Drop missing or invalid geometries
inc00sh = inc00sh.dropna(subset=['geometry'])
inc00sh = inc00sh[inc00sh.geometry.is_valid]



In [ ]:
#merge dataframe
inc16sh= ct16.merge(acs16, left_on= "GEOID", right_on= "GEOID")
inc16sh. head ()

In [ ]:
inc16sh.info ()

In [ ]:
#Drop missing or invalid geometries
inc16sh = inc16sh.dropna(subset=['geometry'])
inc16sh = inc16sh[inc16sh.geometry.is_valid]

In [ ]:
# Plot the adjusted 2000 incomes on a map
labs = {"income_2000_adj": "Median Adjusted Income for 2000"}
fig = px.choropleth_mapbox(inc00sh, geojson=inc00sh.set_index("GEOID"),	locations= "GEOID",
color= "income_2000_adj", opacity= 0.75,
color_continuous_scale= "viridis",
title= "Pennsylvania",
mapbox_style= "carto-positron",
center={"lon":-75.15, "lat": 40.00},
zoom=9.5, height= 600, width= 900, labels= labs)

# Show the plot
fig.show()

In [ ]:
# Plot the adjusted 2016 incomes on a map
labs = {"income_2016": "Median Income for 2016"}
fig = px.choropleth_mapbox(inc16sh, geojson=inc16sh.set_index("GEOID"),	locations= "GEOID",
color= "income_2016", opacity= 0.75,
color_continuous_scale= "viridis",
title= "Pennsylvania",
mapbox_style= "carto-positron",
center={"lon":-75.15, "lat": 40.00},
zoom=9.5, height= 600, width= 900, labels= labs)

# Show the plot
fig.show()

<b> Plot the adjusted 2000 incomes on a map with `vmin=20000` and `vmax=120000`.  Do the same for the 2016 incomes.  Make sure to include the legend (colorbar) and add text to indicate what those color values represent.</b>

We can already see some differences popping out, especially with respect to Center City (and Fishtown) and North Philadelphia.  To properly visulize those, we'd like to take the difference of these two maps.

### **PART 3 - Interpolating the 2000 data on 2016 Locations**

In class, merging the 2000 data into the 2016 GeoDataFrame resulted in a lot of dropped census tracts b/c the definition of the census tracts changed over time.  To recover those, we need an estimate of the adjusted 2000 income value at each 2016 census tract.  We can do this by interpolating (using Gaussian Process Regression [aka Kriging]) the adjusted 2000 values onto the 2016 locations.

<b> Import the GaussianProcessRegressor model as well as the ConstantKernel and RBF kernels. </b>

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

<b> As we did in class, create the features (lat/lon) and target (2000 adjusted median incomes) for the model from the `inc00sh` GeoDataFrame. </b>

In [ ]:
lon00 = inc00sh.centroid.x
lat00 = inc00sh.centroid.y

lon16 = inc16sh.centroid.x
lat16 = inc16sh.centroid.y

fig, ax= plt.subplots(figsize=(8,6))
ax.scatter(lon00, lat00, facecolor= "none", color= "k", s=60, label="2000 centroids")
ax.scatter(lon16, lat16, c= "r", s=20, label="2016 centroids")
ax.legend()
ax.axis("equal")
fig.show ()

As always, the features need to be standardized.

<b> Standardize the features array across objects and standardize the target. </b>

In [ ]:
inc00sh. head (2)

In [ ]:
# Features
feat= pd.DataFrame ()
feat["lon"] = lon00
feat["lat"] = lat00

# Target
targ = inc00sh["income_2000_adj"]


In [ ]:
# Standardize features
feat_st = (feat - feat.mean()) / feat.std()
# Standardize target
targ_st = (targ - targ.mean()) / targ.std()

<b> Fit a GPR model to the standardized 2000 data (remember to intialize the kernel).</b>

In [ ]:
#initialise kernel and fit into data
kernel= C(1.0,(1e-3, 1e3)) * RBF(10., (1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(feat_st, targ_st)

In [ ]:
#create the feature array of prediction
feat_pred=pd.DataFrame ()
feat_pred ["lon"]= lon16
feat_pred ["lat"]= lat16

The model has now been fit to the 2000 adjusted median income data at the 2000 census tract locations.  Now we want to interpolate onto the 2016 positions.

<b> Create features for the 2016 census tract positions making sure to standardize. </b>

In [ ]:
# Standardize target
feat_pred_std= (feat_pred - feat.mean()) / feat.std()

#make prediction
inc16sh_pred= gpr.predict(feat_pred)

<b> Interpolate the trained model onto these positions using the `.predict()` method. </b>

<b> Unstandardize these values so that they are real dollars and put those into a column in `inc16sh` called `"income_2000_interp"`. </b>

In [ ]:
inc16sh. head (2)


In [ ]:
#Unstandardize target
inc16sh["income_2000_interp"]= inc16sh_pred * targ.std() + targ.mean()

#Take the difference between 2016 and "predicted" 2000
inc16sh["inc16sh_diff_pred"]= inc16sh["income_2016"] - inc16sh["ins16sh_pred"]

In [ ]:
inc16sh. head (2)

So now we have a column of interpolated values of the 2000 adjusted incomes at the locations of the 2016 census tracts.

<b> Make a map of the difference between the 2016 median income and the adjusted 2000 median income using the `coolwarm` color map, `vmin=-20000` and `vmax=20000`. </b>

In [ ]:
labs = {"inc16sh_diff_pred": "diff in median 2016 income & adjusted 2000 income"}
fig = px.choropleth_mapbox(inc16sh,
                           geojson=inc16sh.set_index("AFFGEOID"),
locations="AFFGEOID",
color="inc16sh_diff_pred", opacity=0.75,
color_continuous_scale="RdBu",
title="Pennsylvania",
mapbox_style="carto-positron",
center={"lon": -75.15, "lat": 40.00},
zoom=9.5,height=600, width=1200,labels=labs)

# Show the plot
fig.show()

**Describe the spatial similarity between this map and the map of 2000/2016 demographic differences we derived in class.**

Overall, the two sets of visualisations confirm the trend of segregation and median income decline.  In the 2000 and 2016 maps from the class, we see that fraction of white population is gaining ground in South-East and North in particular whereas the non-white fraction is shrinking. Those areas in this map also show the highest difference in income. We also see similarities between the concentration and shrinking of the non-white population in the class map and low median income in this map.